In [1]:
from json import dumps, loads
import time, os
import threading, Queue
import ast
import unittest
import requests
import random
import logging
# chainsapce
from chainspacecontract import transaction_to_solution
# from chainspacecontract.examples.surge import contract as surge_contract
from chainspacecontract.examples import surge
# crypto
from chainspacecontract.examples.utils import setup, key_gen, pack
from chainspaceapi import ChainspaceClient

# Setup logging
logging.basicConfig(level=logging.INFO, filename="execution.log", filemode="a+",
                        format="%(asctime)-15s %(levelname)-8s %(message)s")
logging.info("Starting test_surge_bids")

# Setup variables
r = requests.get('http://10.129.6.52:4999/setup.in')
setup_str = r.text
setup_str = setup_str.split('\n')
NUM_SHARDS = int(setup_str[0])
NUM_REPLICAS = int(setup_str[1])
NUM_CLIENTS = int(setup_str[2])
logging.info("NUM_SHARDS: %s, NUM_REPLICAS: %s, NUM_CLIENTS: %s "%(NUM_SHARDS, NUM_REPLICAS, NUM_CLIENTS))
CS_HOST='10.129.6.52'
G = setup()[0]


In [2]:
global_client = ChainspaceClient(host=CS_HOST,port=5000)
init_transaction = surge.init()
init_tokens = init_transaction['transaction']['outputs']
global_client.process_transaction(init_transaction)
client_divs = surge.eq_div(NUM_CLIENTS, NUM_SHARDS)
print client_divs

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": [], "methodID": "init", "parameters": [], "outputs": ["{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
[150, 150]


In [3]:
# Create prosumer clients for locations 
clients = []
base_port=5000
sreps = []

for s in range(0,NUM_SHARDS):
    clients.append([])
    sreps.append(surge.SREPClient(host=CS_HOST, port=base_port+s))
    
    for c in range(0,client_divs[s]):
        clients[s].append(surge.SurgeClient(host=CS_HOST, port=base_port+s))

print clients

threads = []
for s in range(0,NUM_SHARDS):
    threads.append([])
    for c in range(0,client_divs[s]):
        client = clients[s][c]
        idx = sum(client_divs[:s]) + c
        threads[s].append(threading.Thread(target=client.create_surge_client, args=(init_tokens[idx],)) )

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].start()

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].join()

[[<chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce515d40>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce515cf8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce515d88>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce515dd0>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce515e18>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce515e60>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce515ea8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce515ef0>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce515f38>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce515f80>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce515fc8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71ce4d0098>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f71cf4da950>, <chainspac

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_84dcdb21f756ee3eed6caa71a1d86f7aba06a53e3ab07bd9f9bb4949b2f8336d"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd02c809f1b7325ce1c655477ca23afe94e45e9534782ee6b7ee04310684", "92c71d002bbeb33dd1ad1d4d1133282498cfcfbeb2a739a1a941c3c317b4ce6ab9c71d002b3cb2b1ecbf9493a5f360227bd20a14775f522b870fcf339297e82b84"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02c809f1b7325ce1c655477ca23afe94e45e9534782ee6b7ee04310684\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02c809f1b7325ce1c655477ca23afe94e45e9534782ee6b7ee04310684\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02c809f1b7325ce1c655477ca23afe94e45e9534782ee6b7ee04310684\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_84dcdb21f756ee3eed6caa71a1d86f7aba06a53e3ab07bd9f9b

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_b1d499737d577f0fa7e8dc797daee9942975ba1becc5fe448b7ca2b3df4ed9a0"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd02e7ce912c37fcf468f6346a8b6e34a8d06d931facc77bbec3da362555", "92c71d002ba08a7e5c56c0cfc80f95759e78488f1d045d61ca6b2e7d77aab5291ac71d002bd7e8554c1733044327aaced89537822883482f760e7b40b0954f3a96"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02e7ce912c37fcf468f6346a8b6e34a8d06d931facc77bbec3da362555\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02e7ce912c37fcf468f6346a8b6e34a8d06d931facc77bbec3da362555\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02e7ce912c37fcf468f6346a8b6e34a8d06d931facc77bbec3da362555\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_b1d499737d577f0fa7e8dc797daee9942975ba1becc5fe448b7

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_75ae35bd41a63868ed4b35ecb48cebbdb381bc602280ee43a3f43f50729e6b87"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd03dc8a91486c4d542a53e3e40a87ed68c6f2df0b0c3d18d6207dab16a9", "92c71d002b3ad509cb3666d7d5a9ef2c8cd3ac575206479b183f2d61e81b756eaac71d002b078684c425fadcaaed643baea86bbde4212849303b6b4279002309fd"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03dc8a91486c4d542a53e3e40a87ed68c6f2df0b0c3d18d6207dab16a9\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03dc8a91486c4d542a53e3e40a87ed68c6f2df0b0c3d18d6207dab16a9\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03dc8a91486c4d542a53e3e40a87ed68c6f2df0b0c3d18d6207dab16a9\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_75ae35bd41a63868ed4b35ecb48cebbdb381bc602280ee43a3f

{"transaction": {"inputIDs": ["0_55121a675b33c7458a4a8dd9caac2191e32b8f243b4a7eda7878338f220cb9fa"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd0245a8d4754bb128a25b9b1b84c7247e87a043263f8a7561b1a2650533", "92c71d002b3eac2976a856c255ccf0b4fcdf06d60678c8fb199999d8495807649ac71d002b6bf27481ffa65767713b07d49f78cf76ba8b290678e86f10865633da"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0245a8d4754bb128a25b9b1b84c7247e87a043263f8a7561b1a2650533\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0245a8d4754bb128a25b9b1b84c7247e87a043263f8a7561b1a2650533\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0245a8d4754bb128a25b9b1b84c7247e87a043263f8a7561b1a2650533\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_55121a675b33c7458a4a8dd9caac2191e32b8f243b4a7eda7878338f220cb9fa": "{\"type\": \"InitToken\", \"location\": 0}"}}
POS

...

{"transaction": {"inputIDs": ["1_40f40960248490b50fae8cc3750122a9c4495c7837ad5065a5f176cc391f648d"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd02a9ddeec7539cbe7b2a47cec715f36fb27cc760efdfc7762df86f617b", "92c71d002be4fae3dc243183328c77f069cb7e58a32338408a0d7e813402a1d33dc71d002b4c503d93df65c03b929a653fe9f87aa9030ab9604140e3b3c8642627"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 1, \"pub\": \"c7220292cd02c9bd02a9ddeec7539cbe7b2a47cec715f36fb27cc760efdfc7762df86f617b\"}", "{\"type\": \"VoteSlipToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd02a9ddeec7539cbe7b2a47cec715f36fb27cc760efdfc7762df86f617b\"}", "{\"type\": \"EBToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd02a9ddeec7539cbe7b2a47cec715f36fb27cc760efdfc7762df86f617b\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"1_40f40960248490b50fae8cc3750122a9c4495c7837ad5065a5f176cc391f648d": "{\"type\": \"InitToken\", \"location\": 1}"}}POST

POST http://10.129.6.52:5001/api/1.0/transaction/process HTTP/1.1POST http://10.129.6.52:5001/api/1.0/transaction/process HTTP/1.1

{"transaction": {"inputIDs": ["1_01faf20c8f1b996c62cb5b60cd0a7b5921e60768e622d4c6c5baa892032e36b9"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd03e7e8a2787b88b915aada8f756ca69052dd9ec00ebcfe55a0cbd8011d", "92c71d002b93eb2bb7df270a12430dad9d5a359109b5e8e9561640abc4fbd5f46ac71d002bb8c514369611c4aa9252e69b938c48b5e7c77b6530517f5fd04be04b"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 1, \"pub\": \"c7220292cd02c9bd03e7e8a2787b88b915aada8f756ca69052dd9ec00ebcfe55a0cbd8011d\"}", "{\"type\": \"VoteSlipToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd03e7e8a2787b88b915aada8f756ca69052dd9ec00ebcfe55a0cbd8011d\"}", "{\"type\": \"EBToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd03e7e8a2787b88b915aada8f756ca69052dd9ec00ebcfe55a0cbd8011d\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OKHTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}

{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}


In [5]:
# Create threads for casting srep vote

threads = []
queues = []

for s in range(0,NUM_SHARDS):
    que = Queue.Queue()
    threads.append([])
    for c in range(0,client_divs[s]):
        t = threading.Thread(target=clients[s][c].cast_srep_vote, args=(sreps[s].pub, que)) 
        threads[s].append(t)
    queues.append(que)

# Run bidding threads
start = time.time()
for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].start()

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].join()

# Submit the votes
threads = []
for s in range(0,NUM_SHARDS):
    votes = []
    que = queues[s]
    while not que.empty():
        votes.append(que.get())
    client = sreps[s]
    t = threading.Thread(target=client.create_srep_client, args=(base_port+s, tuple(votes)) )
    threads.append(t)
    
for t in threads:
    t.start()
    
for t in threads:
    t.join()

end = time.time()
duration = end-start
logging.info("Execution took "+ str(duration))
logging.info("TPS: %s"%(NUM_CLIENTS/duration))

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
 {"transaction": {"inputIDs": ["0_d9e03053dd1b0832723c64abde6d742c640e3cdbe269e6c7a3d6a3dc27d6d60c"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b14b9836525565a1c73e31bc582f6eb0bcfe9c611ce0f2f4fdfb019c8c71d002be2a1eb9506b2611a5491400ba19939b716033b256bd1dd41311c8963"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd020c6f6e452fa234df0cbb9abdd229aba071f812cc9b535b0c7f9e1bd1\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd0277e0c96dcf8c469cd4d98a652ecce7664933f8bad7a50da39b965fbf\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0277e0c96dcf8c469cd4d98a652ecce7664933f8bad7a50da39b965fbf\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_d9e03053dd1b0832723c64abde6d742c640e3cdbe269e6c7a3d6a3dc27d6d60c": "{\"type\": \"VoteSlipToken\

{"transaction": {"inputIDs": ["0_ba4b6d23daf1101de1931a304eda9adafdbe36175d9f3f8363be7c0b31abb9b4"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bdd210e83dd5558df717fd003b76fb4cfb5c98118a8c216b207906890c71d002b2cf261ac887f1d48a1788a4a7fd74feca4ffdef9b3b8ea6beb60fb30"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd020c6f6e452fa234df0cbb9abdd229aba071f812cc9b535b0c7f9e1bd1\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd03f41229cf4d16bfe2e6106a5b129fa8994841c980e2d1630fa301247f\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03f41229cf4d16bfe2e6106a5b129fa8994841c980e2d1630fa301247f\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_ba4b6d23daf1101de1931a304eda9adafdbe36175d9f3f8363be7c0b31abb9b4": "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03f41229cf4d16bfe2e6106a5b129fa8994841c980e2d1630fa301247f\"}"}}
POST http://10.129.6.

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_ab78663b02140df923fc47fb95bdce78322b9a46b0b7ba3cab38a03829800b32"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b230464fd8d49bd5333f4c223b93d8987ba49659b589ed0963f783c94c71d002b88e266476091693f9d06941d1ef00d848f961ac66eb66d4ea4d9086c"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd020c6f6e452fa234df0cbb9abdd229aba071f812cc9b535b0c7f9e1bd1\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02e7ce912c37fcf468f6346a8b6e34a8d06d931facc77bbec3da362555\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02e7ce912c37fcf468f6346a8b6e34a8d06d931facc77bbec3da362555\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_ab78663b02140df923fc47fb95bdce78322b9a46b0b7ba3cab38a03829800b32": "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02e7ce912c37fcf468f6

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_42152e72fc66ca0ba65933e4124bc372cf4aef739fc896f60c42a485b86f5ab0"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b0f3161da145caa2d7989a13413c8de66540be762166f12e28b7bb075c71d002bfda9a774ba2afccac267124cbba67ff0f56c4c47c41deca942fef8f5"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd020c6f6e452fa234df0cbb9abdd229aba071f812cc9b535b0c7f9e1bd1\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd03ec053617ac00b5d1dbf1de1bfbcf818b4b65de7d3eec90d2c08b1fae\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03ec053617ac00b5d1dbf1de1bfbcf818b4b65de7d3eec90d2c08b1fae\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_42152e72fc66ca0ba65933e4124bc372cf4aef739fc896f60c42a485b86f5ab0": "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03ec053617ac00b5d1db

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
 POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1HTTP/1.1 200 OKPOST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_4ef2265d39a1fb13bf87354f2576c6e3f56571d42d9786014d40b1e15b25fec7"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bc9ef76278adceb34f17d29e8e630fd15f8f498167a25cff3a5fa8b55c71d002b194161d6847e57b6b82ef364a776caf8c64b855bffcb65bef95399b1"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd020c6f6e452fa234df0cbb9abdd229aba071f812cc9b535b0c7f9e1bd1\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02186bd0b5fca02c9dce96e63b17d3ae50675ebdced98b0f80616c28fe\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02186bd0b5fca02c9dce96e63b17d3ae50675ebdced98b0f80616c28fe\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_4ef2265d39a1fb1

...

POST http://10.129.6.52:5001/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["1_f910f5ef53d29976570ae237fe3f93eea2bbb235899b57d562121c2881588ea3"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bf6dd3b8bfa074b60300c1b913381c2a93241518159c2c7c94682868cc71d002bf18f459c99ed71b60461f323add01df98765ee167b6ab3ed19016d46"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02863897d387cf4ec1dc4a3b9f7b8b5254a7e6994104b3b2efbeafce87\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd0327e2b4de5fd0e27da2575bcbb383ac33c046ae45ab35c1b3f7107211\", \"location\": 1}", "{\"type\": \"VoteSlipToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd0327e2b4de5fd0e27da2575bcbb383ac33c046ae45ab35c1b3f7107211\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"1_f910f5ef53d29976570ae237fe3f93eea2bbb235899b57d562121c2881588ea3": "{\"type\": \"VoteSlipToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd0327e2b4de5fd0e27da2

POST http://10.129.6.52:5001/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["1_d46b52f7802390ab7e3a844200e16cb142305046ce7de65b0b4df55d6c50150b"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bea79bcf8ccf32025027ab6cfdedb29c2e70f01a272b6ab13d2a3f162c71d002ba0fc0768ca51e07b6ddb359f9e1e23fb0943d4b0a568dbae08788f6f"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02863897d387cf4ec1dc4a3b9f7b8b5254a7e6994104b3b2efbeafce87\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd03a3a23dc500df06fa8480406e144e3e84c3e3f6e5ad9e0ba18af44d48\", \"location\": 1}", "{\"type\": \"VoteSlipToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd03a3a23dc500df06fa8480406e144e3e84c3e3f6e5ad9e0ba18af44d48\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"1_d46b52f7802390ab7e3a844200e16cb142305046ce7de65b0b4df55d6c50150b": "{\"type\": \"VoteSlipToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd03a3a23dc500df06fa84

POST http://10.129.6.52:5001/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["1_9cb0878a340d33dc81b0f2fed4aaf92333a211bed75288a84172d4acfeb9298c"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b3dd8d05eed9d9cc1205970912e7fe35469df3c156ac566e42cc275e4c71d002b0ea6dc36fda4e31972056d55d3ed5462aed7982ae6d82623ed143a03"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02863897d387cf4ec1dc4a3b9f7b8b5254a7e6994104b3b2efbeafce87\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02a2a12ce85e1bc87bd4098bce88953c5fb1cd4529f38b5ce42d10d8c5\", \"location\": 1}", "{\"type\": \"VoteSlipToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd02a2a12ce85e1bc87bd4098bce88953c5fb1cd4529f38b5ce42d10d8c5\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"1_9cb0878a340d33dc81b0f2fed4aaf92333a211bed75288a84172d4acfeb9298c": "{\"type\": \"VoteSlipToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd02a2a12ce85e1bc87bd4

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OKHTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}

{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
